In [14]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [15]:
data = pd.read_csv('../data/raw/ml-100k/u.data', sep = '\t', header = None)
data.columns = ['user_id', 'item_id', 'rating', 'timestamp']

info = pd.read_csv('../data/raw/ml-100k/u.info', header = None)
item = pd.read_csv('../data/raw/ml-100k/u.item', sep = '|', header = None, encoding='latin-1')
genres = pd.read_csv('../data/raw/ml-100k/u.genre', sep = '|', header = None)

user = pd.read_csv('../data/raw/ml-100k/u.user', sep = '|', header = None)
user.columns = ['user_id', 'age', 'gender', 'occupation', 'zip code']

occupations = pd.read_csv('../data/raw/ml-100k/u.occupation', sep = '|', header = None)


In [16]:
info = {
    'users': info[0][0].split()[0],
    'items': info[0][1].split()[0],
    'ratings': info[0][2].split()[0]
}

In [17]:
genre_columns = []
for genre in genres[0]:
    genre_columns.append(genre)

In [18]:
item.columns = ['item_id', 'movie title', 'release date', 'video release date',
              'IMDb URL'] + genre_columns

In [19]:
occupation_columns = []
for occupation in occupations[0]:
    occupation_columns.append(occupation)

In [20]:
movie_rating_matrix = data.pivot_table(index=['user_id'], columns=['item_id'], values=['rating'], fill_value=0.0)
movie_rating_matrix = movie_rating_matrix.reset_index()
movie_rating_matrix

user_id rating                          ...                           \
item_id              1  2  3  4  5  6  7  8  9  ... 1673 1674 1675 1676 1677   
0             1      5  3  4  3  3  5  4  1  5  ...    0    0    0    0    0   
1             2      4  0  0  0  0  0  0  0  0  ...    0    0    0    0    0   
2             3      0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0   
3             4      0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0   
4             5      4  3  0  0  0  0  0  0  0  ...    0    0    0    0    0   
..          ...    ... .. .. .. .. .. .. .. ..  ...  ...  ...  ...  ...  ...   
938         939      0  0  0  0  0  0  0  0  5  ...    0    0    0    0    0   
939         940      0  0  0  2  0  0  4  5  3  ...    0    0    0    0    0   
940         941      5  0  0  0  0  0  4  0  0  ...    0    0    0    0    0   
941         942      0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0   
942         943      0  5  0  0  0  0  0  0  3  ...    0    0    0    0    0   

                                  
item_id 1678 1679 1680 1681 1682  
0          0    0    0    0    0  
1          0    0    0    0    0  
2          0    0    0    0    0  
3          0    0    0    0    0  
4          0    0    0    0    0  
..       ...  ...  ...  ...  ...  
938        0    0    0    0    0  
939        0    0    0    0    0  
940        0    0    0    0    0  
941        0    0    0    0    0  
942        0    0    0    0    0  

[943 rows x 1683 columns]

In [21]:
movie_genre_table = item.copy()
movie_genre_table.drop(['movie title', 'release date', 'video release date', 'IMDb URL'], axis=1, inplace=True)
movie_genre_table

,item_id,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1678,1679,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1679,1680,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1680,1681,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [22]:
user = user.assign(tmp=[1 for i in range(user.shape[0])])
user['gender'] = [1 if gen == "M" else 0 for gen in user['gender']]

In [31]:
users_occupation_table = pd.pivot_table(user, index=['user_id', 'age', 'gender'], columns='occupation', values=['tmp'], fill_value=0)['tmp']
users_occupation_table = users_occupation_table.reset_index()
scaler = StandardScaler()

users_occupation_table['age'] = scaler.fit_transform(np.array(users_occupation_table['age']).reshape(-1, 1))
users_occupation_table

occupation,user_id,age,gender,administrator,artist,doctor,educator,engineer,entertainment,executive,...,marketing,none,other,programmer,retired,salesman,scientist,student,technician,writer
0,1,-0.824859,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,2,1.554867,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,3,-0.906919,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,4,-0.824859,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,5,-0.086324,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,939,-0.660740,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
939,940,-0.168383,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
940,941,-1.153098,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
941,942,1.144570,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
X_train_5 = pd.read_csv('../data/raw/ml-100k/u5.base', sep='\t')
X_test_5 = pd.read_csv('../data/raw/ml-100k/u5.test', sep='\t')
X_train_4 = pd.read_csv('../data/raw/ml-100k/u4.base', sep='\t')
X_test_4 = pd.read_csv('../data/raw/ml-100k/u4.test', sep='\t')
X_train_3 = pd.read_csv('../data/raw/ml-100k/u3.base', sep='\t')
X_test_3 = pd.read_csv('../data/raw/ml-100k/u3.test', sep='\t')
X_train_2 = pd.read_csv('../data/raw/ml-100k/u2.base', sep='\t')
X_test_2 = pd.read_csv('../data/raw/ml-100k/u2.test', sep='\t')
X_train_1 = pd.read_csv('../data/raw/ml-100k/u1.base', sep='\t')
X_test_1 = pd.read_csv('../data/raw/ml-100k/u1.test', sep='\t')
X_train_a = pd.read_csv('../data/raw/ml-100k/ua.base', sep='\t')
X_test_a = pd.read_csv('../data/raw/ml-100k/ua.test', sep='\t')
X_train_b = pd.read_csv('../data/raw/ml-100k/ub.base', sep='\t')
X_test_b = pd.read_csv('../data/raw/ml-100k/ub.test', sep='\t')

columns = ['user_id', 'item_id', 'ratings', 'timestamp']
X_train_1.columns = columns
X_train_1.drop(['timestamp'], axis=1, inplace=True)
X_train_2.columns = columns
X_train_2.drop(['timestamp'], axis=1, inplace=True)
X_train_3.columns = columns
X_train_3.drop(['timestamp'], axis=1, inplace=True)
X_train_4.columns = columns
X_train_4.drop(['timestamp'], axis=1, inplace=True)
X_train_5.columns = columns
X_train_5.drop(['timestamp'], axis=1, inplace=True)
X_train_a.columns = columns
X_train_a.drop(['timestamp'], axis=1, inplace=True)
X_train_b.columns = columns
X_train_b.drop(['timestamp'], axis=1, inplace=True)

X_test_1.columns = columns
X_test_1.drop(['timestamp'], axis=1, inplace=True)
X_test_2.columns = columns
X_test_2.drop(['timestamp'], axis=1, inplace=True)
X_test_3.columns = columns
X_test_3.drop(['timestamp'], axis=1, inplace=True)
X_test_4.columns = columns
X_test_4.drop(['timestamp'], axis=1, inplace=True)
X_test_5.columns = columns
X_test_5.drop(['timestamp'], axis=1, inplace=True)
X_test_a.columns = columns
X_test_a.drop(['timestamp'], axis=1, inplace=True)
X_test_b.columns = columns
X_test_b.drop(['timestamp'], axis=1, inplace=True)

X_train_1['ratings'] = scaler.fit_transform(np.array(X_train_1['ratings']).reshape(-1, 1))
X_train_2['ratings'] = scaler.fit_transform(np.array(X_train_2['ratings']).reshape(-1, 1))
X_train_3['ratings'] = scaler.fit_transform(np.array(X_train_3['ratings']).reshape(-1, 1))
X_train_4['ratings'] = scaler.fit_transform(np.array(X_train_4['ratings']).reshape(-1, 1))
X_train_5['ratings'] = scaler.fit_transform(np.array(X_train_5['ratings']).reshape(-1, 1))
X_train_a['ratings'] = scaler.fit_transform(np.array(X_train_a['ratings']).reshape(-1, 1))
X_train_b['ratings'] = scaler.fit_transform(np.array(X_train_b['ratings']).reshape(-1, 1))

X_test_1['ratings'] = scaler.fit_transform(np.array(X_test_1['ratings']).reshape(-1, 1))
X_test_2['ratings'] = scaler.fit_transform(np.array(X_test_2['ratings']).reshape(-1, 1))
X_test_3['ratings'] = scaler.fit_transform(np.array(X_test_3['ratings']).reshape(-1, 1))
X_test_4['ratings'] = scaler.fit_transform(np.array(X_test_4['ratings']).reshape(-1, 1))
X_test_5['ratings'] = scaler.fit_transform(np.array(X_test_5['ratings']).reshape(-1, 1))
X_test_a['ratings'] = scaler.fit_transform(np.array(X_test_a['ratings']).reshape(-1, 1))
X_test_b['ratings'] = scaler.fit_transform(np.array(X_test_b['ratings']).reshape(-1, 1))



In [39]:
X_train_2

,user_id,item_id,ratings
0,1,4,-0.468200
1,1,5,-0.468200
2,1,6,1.310483
3,1,7,0.421142
4,1,10,-0.468200
...,...,...,...
79994,943,1067,-1.357541
79995,943,1074,0.421142
79996,943,1188,-0.468200
79997,943,1228,-0.468200


In [34]:
X_train_1.to_csv('../data/interim/X_train_1.csv')
X_train_2.to_csv('../data/interim/X_train_2.csv')
X_train_3.to_csv('../data/interim/X_train_3.csv')
X_train_4.to_csv('../data/interim/X_train_4.csv')
X_train_5.to_csv('../data/interim/X_train_5.csv')
X_train_a.to_csv('../data/interim/X_train_6.csv')
X_train_b.to_csv('../data/interim/X_train_7.csv')

X_test_1.to_csv('../data/interim/X_test_1.csv')
X_test_2.to_csv('../data/interim/X_test_2.csv')
X_test_3.to_csv('../data/interim/X_test_3.csv')
X_test_4.to_csv('../data/interim/X_test_4.csv')
X_test_5.to_csv('../data/interim/X_test_5.csv')
X_test_a.to_csv('../data/interim/X_test_6.csv')
X_test_b.to_csv('../data/interim/X_test_7.csv')

In [35]:
data.to_csv("../data/interim/data.csv")
pd.DataFrame([info]).to_csv("../data/interim/info.csv")
item.to_csv("../data/interim/item.csv")
pd.DataFrame(genres).to_csv("../data/interim/genres.csv")
user.to_csv("../data/interim/user.csv")
pd.DataFrame(occupations).to_csv("../data/interim/occupations.csv")
movie_genre_table.to_csv('../data/interim/movie_genre_table.csv')
movie_rating_matrix.to_csv('../data/interim/movie_rating_matrix.csv')
users_occupation_table.to_csv('../data/interim/user_occupation_table.csv')